# 并行编程的惯用法


人们通常认为并行等同于多核,但现代计算机在不同层次上都使用了并行技术.比如说,单核的运行速度现今仍能每年不断提升的原因是--单核包含的晶体管数量,如同摩尔定律预测的那样变得越来越多,而单核在位级和指令级两个层次上都能够并行地使用这些晶体管资源.

+ 位级(bit-level)并行

    为什么32位计算机的运行速度比8位计算机更快？因为并行.对于两个32位数的加法,8位计算机必须进行多次8位计算,而32位计算机可以一步完成,即并行地处理32位数的4字节.

    计算机的发展经历了8位、16位、32位,现在正处于64位时代.然而由位升级带来的性能改善是存在瓶颈的,这也正是短期内我们无法步入128位时代的原因.

+ 指令级(instruction-level)并行

    现代CPU的并行度很高,其中使用的技术包括流水线、乱序执行和猜测执行等.

    程序员通常可以不关心处理器内部并行的细节,因为尽管处理器内部的并行度很高,但是经过精心设计,从外部看上去所有处理都像是串行的.

    而这种"看上去像串行"的设计逐渐变得不适用.处理器的设计者们为单核提升速度变得越来越困难.进入多核时代,我们必须面对的情况是--无论是表面上还是实质上,指令都不再串行执行了.

+ 数据级(data)并行

    数据级并行(也称为"单指令多数据",SIMD)架构,可以并行地在大量数据上施加同一操作.这并不适合解决所有问题,但在适合的场景却可以大展身手.

    图像处理就是一种适合进行数据级并行的场景.比如为了增加图片亮度就需要增加每一个像素的亮度.现代GPU(图形处理器)也因图像处理的特点而演化成了极其强大的数据并行处理器.

+ 任务级(task-level)并行

    终于来到了大家所认为的并行形式——多处理器.从程序员的角度来看,多处理器架构最明显的分类特征是其内存模型(共享内存模型或分布式内存模型).

    + 对于共享内存的多处理器系统,每个处理器都能访问整个内存,处理器之间的通信主要通过内存进行.

    + 对于分布式内存的多处理器系统,每个处理器都有自己的内存,处理器之间的通信主要通过网络进行.

    通过内存通信比通过网络通信更简单更快速,所以用共享内存编程往往更容易.然而当处理器个数逐渐增多,共享内存就会遭遇性能瓶颈——此时不得不转向分布式内存.如果要开发一个容错系统,就要使用多台计算机以规避硬件故障对系统的影响,此时也必须借助于分布式内存.

## 并行方式如何选择

首先python的多线程和协程都是共享内存式的,而多进程虽然是在同一机器上,但各个进程间并不共享内存,因此是分布式的.

看起来多进程,多线程,协程都是以并行的方式运行的,那么我们该如何选择使用什么技术呢?

+ 首先我们可以简单的通过分析目标功能来选择,如果我们的项目主要是计算密集型的,比如是并行计算多个数据是否是质数这类,那么没得选,只有多进程才可以做到最大化利用cpu资源,另外两个都只能跑满一个cpu核心.

+ 接着就是主要是io操作的任务了,io密集型任务首选当然是协程,也只有协程可以搞定[10k](http://www.kegel.com/c10k.html)问题,但python的默认I/O多是同步I/O,因此在所需依赖无法满足的情况下只能使用多线程方式替代.

协程和多线程都最多跑满一个核心,但其机制是完全不一样的,协程是用户组织代码,因此是写成顺序执行的异步执行,说白了还是在顺序执行,只是线程运行哪段代码会在协程间跳转执行,打个比方有点像拉链,只要有一个齿坏了,整个过程就会卡住.

但多线程则完全不同,一个线程卡死了并不会影响其他线程.

## 并行编程的常用同步机制(原语)

并行程序由于并行会带来对资源的争抢从而影响执行的正确性,因此需要在用到共享资源时进行同步,从而在局部位置降低并行程度,避免争抢资源以确保执行正确.同步意味着降低并行度,自然就会影响执行效率,因此同步机制可以理解为用效率换正确执行.

同步机制的具体实现被称为同步原语,python的每一种并行机制都包含多种同步原语,这些原语使用思路上是一致的,接口也基本一致,只是使用的模块会有些许不同.

### 信号量Semaphore

信号量同步基于内部计数器,每调用一次acquire(),计数器减1;每调用一次release(),计数器加1.计数器的值永远不会小于0.当计数器为0时.acquire()调用被阻塞,直到其他线程来调用release().Semaphore支持上下文管理协议

Semaphore的接口有两个:

+ `acquire()` 获取一个信号量,协程中这个方法是一个协程
+ `release()` 释放一个信号量
+ **`locked()` 协程中独有,用来判断是否被锁定**


信号量有两种:

+ Semaphore 

    标准信号量
    
+ BoundedSemaphore

    有界信号量,它会检查内部计数器的值,并保证它不会大于初始值,如果超了,就引发一个`ValueError`
    
多数情况下,semaphore用于控制并行量,如果`semaphore`被`release()`过多次,这意味着存在bug.


信号量在线程,进程,协程中的使用的模块并不一样:

+ 协程--`asynico.Semaphore(value=1)`

+ 线程--`threading.Semaphore(value=1)`

+ 进程--`multiprocessing.Semaphore([value])`

> 协程版本

In [53]:
import time
import asyncio
from random import random

sema = asyncio.Semaphore(3)

async def foo(cid):
    async with sema:
        print(f'{cid} acquire sema')
        wt = random() * 2
        time.sleep(wt)
    print(f'{cid} release sema')

async def main():
    for task in asyncio.as_completed([foo(num) for num in range(5)]):
        await task

await main()

2 acquire sema
2 release sema
3 acquire sema
3 release sema
4 acquire sema
4 release sema
0 acquire sema
0 release sema
1 acquire sema
1 release sema


> 多线程版本

In [55]:
import time
from random import random
from threading import Thread, Semaphore
sema = Semaphore(3)
def foo(tid):
    with sema:
        print(f'{tid} acquire sema')
        wt = random() * 2
        time.sleep(wt)
    print(f'{tid} release sema')

def main():
    threads = []
    for i in range(5):
        t = Thread(target=foo, args=(i,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()

main()

0 acquire sema1 acquire sema
2 acquire sema

0 release sema3 acquire sema

2 release sema
4 acquire sema
1 release sema
3 release sema
4 release sema


> 多进程版本

In [56]:
%%writefile src/semaphore.py
from multiprocessing import Process, Semaphore

def foo(tid,sema):
    import time
    from random import random
    with sema:
        print(f'{tid} acquire sema')
        wt = random() * 2
        time.sleep(wt)
    print(f'{tid} release sema')
    
if __name__ == "__main__":
    sema = Semaphore(3)
    processes = []
    for i in range(5):
        t = Process(target=foo, args=(i,sema))
        processes.append(t)

    for t in processes:     
        t.start()
    for t in processes:
        t.join()


Overwriting src/semaphore.py


In [57]:
!python src/semaphore.py

0 acquire sema
1 acquire sema
3 acquire sema
0 release sema
2 acquire sema
2 release sema
4 acquire sema
3 release sema
4 release sema
1 release sema


### 锁Lock

Lock也可以叫做互斥锁,其实相当于信号量为1.

在多线程中锁的作用是用于锁定读写,以确认同一个资源同一时间只能被一个操作访问.

python中锁有两种

+ Lock

    标准锁

+ RLock

    可重入锁,可以由同一个过程多次获取.在内部,除了原始锁使用的锁定/解锁状态之外,它还使用"拥有过程"和"递归级别"的概念.在锁定状态下,某些过程拥有锁;在解锁状态下,没有线程拥有它.
    
锁在线程,进程,协程中的使用的模块并不一样:

+ 协程--`asyncio.Lock()`

+ 线程--`threading.Lock()`/`threading.RLock()`

+ 进程--`multiprocessing.Lock()`/`multiprocessing.RLock()`
    
通常锁并不会单独使用会绑定全局变量或其他公共资源.在并行编程中为了防止不同的过程(线程/进程/协程)同时对一个公用的资源进行修改,我们就会为这个资源设置对应的锁.

我们先看一个不加锁的例子:

In [6]:
import time
from threading import Thread

value = 0

def getlock():
    global value
    new = value + 1
    time.sleep(0.001)  # 使用sleep让线程有机会切换
    value = new
    
threads = []
for i in range(100):
    t = Thread(target=getlock)
    t.start()
    threads.append(t)
for t in threads:
    t.join()
print(value)


9


In [7]:
import time
from threading import Thread, Lock

value = 0
lock = Lock()

def getlock():
    global value
    with lock:
        new = value + 1
        time.sleep(0.001)
        value = new

threads = []
for i in range(100):
    t = Thread(target=getlock)
    t.start()
    threads.append(t)
for t in threads:
    t.join()
print(value)

100


不加锁的情况下,结果会远远的小于100.那我们加上互斥锁看看

锁通常都用于限制共享资源的访问,比如多个流程处理同一个文件,就需要加锁来限制

> 协程版本

In [17]:
import asyncio
import aiofiles

lock = asyncio.Lock()

async def worker_with(lock, f, cid):  
    async with lock:  
        async with aiofiles.open(f,"a+") as fs:
            await fs.write(f'Lock acquired via with {cid}\n')
        print(f"Lock acquired via with {cid}")

async def main():
    f = "source/file.txt"
    async with aiofiles.open(f,"w") as fs:
        await fs.write("")
        
    for task in asyncio.as_completed([worker_with(lock, f,i) for i in range(5)]):
        await task
        
await main()

Lock acquired via with 4
Lock acquired via with 0
Lock acquired via with 2
Lock acquired via with 1
Lock acquired via with 3


> 多线程版本

In [14]:
import sys
import threading
  
def worker_with(lock, f, tid):  
    with lock:  
        with open(f,"a+") as fs:
            fs.write(f'Lock acquired via with {tid}\n')
        print(f"Lock acquired via with {tid}")

def main():
    f = "source/file.txt"
    with open(f,"w") as fs:
        fs.write("")
    lock = threading.Lock()  
    threads = []
    for i in range(5):
        t = threading.Thread(target=worker_with, args=(lock, f,i))
        threads.append(t)
    for t in threads:     
        t.start()
    for t in threads:
        t.join()

main()

Lock acquired via with 0
Lock acquired via with 1
Lock acquired via with 2
Lock acquired via with 3
Lock acquired via with 4


> 多进程版本

而针对于多进程,锁同样起到一个全局信号的作用,比如多个进程处理同一个文件,就需要加锁来限制

In [11]:
%%writefile src/lock.py
import multiprocessing  
import sys  
  
def worker_with(lock, f, pid):  
    with lock:  
        with open(f,"a+") as fs:
            fs.write(f'Lock acquired via with {pid}\n')
        print(f"Lock acquired via with {pid}")

if __name__ == '__main__':
    f = "source/file.txt"
    with open(f,"w") as fs:
        fs.write("")
    lock = multiprocessing.Lock()  
    processes = []
    for i in range(5):
        t = multiprocessing.Process(target=worker_with, args=(lock, f,i))
        processes.append(t)
    for t in processes:     
        t.start()
    for t in processes:
        t.join()

Overwriting src/lock.py


In [12]:
!python src/lock.py

Lock acquired via with 0
Lock acquired via with 2
Lock acquired via with 1
Lock acquired via with 4
Lock acquired via with 3


### 事件

一个过程发送/传递事件,所谓事件是指的一个保存标记状态的对象,如果内部标记为True则表示事件发生了,反之就是没发生

事件的接口包括:

+ `clear()` 事件内部标记为`False`

+ `is_set()` 返回事件的内部标记

+ `set()` 调用则设置内部标记为`True`

+ `wait()` 等待事件被标记为`True`,协程中该接口为协程.

事件在线程,进程,协程中的使用的模块并不一样:

+ 协程--`asyncio.Event()`

+ 线程--`threading.Event()`

+ 进程--`multiprocessing.Event()`


信号量通常用于传递信号触发特定操作.由于一个事件的变化所有的`wait`都可以感知,因此也是程序内部广播的最简单实现方式.一般是拉一个旁路的控制协程/线程/进程阻塞在那边等待特定事件被设置.一旦被设置则会进入到后续逻辑执行特定操作,比如控制主流程启动,暂停等.我们用<生产者/消费者>模型的例子.


> 协程版本

In [60]:
import random
import asyncio

CONSUMER_COUNT = 4

async def consumer(evt):
    t = asyncio.current_task()
    await evt.wait()  # wait()方法创建了一个名为waiter的锁，
    print(f'{t.get_name()}: Resource is available to consumer')
        
async def producer(evt):
    t = asyncio.current_task()
    print(f'{t.get_name()}: wait 2s')
    await asyncio.sleep(2)
    print(f'{t.get_name()}: Making resource available')
    evt.set()  # 设置事件唤醒消费者
        
async def main():
    evt = asyncio.Event()
    cors = []
    for i in range(CONSUMER_COUNT):
        c = asyncio.create_task(consumer(evt), name=f'c{i}')
        cors.append(c)
    
    p = asyncio.create_task(producer(evt), name='p')
    cors.append(p)
    for task in asyncio.as_completed(cors):
        await task

await main()

p: wait 2s
p: Making resource available
c0: Resource is available to consumer
c1: Resource is available to consumer
c2: Resource is available to consumer
c3: Resource is available to consumer


> 多线程版本

In [66]:
import time
import threading
from random import randint

TIMEOUT = 3
CONSUMER_COUNT = 4

def consumer(evt):
    t = threading.current_thread()
    evt.wait(TIMEOUT)
    print(f'{t.name}: Resource is available to consumer')
    
def producer(evt):
    t = threading.current_thread()
    print(f'{t.name}: wait 2s')
    time.sleep(2)
    print(f'{t.name}: Making resource available')
    evt.set()  # 设置事件唤醒消费者
    
def main():
    evt = threading.Event()
    threads = []
    for i in range(CONSUMER_COUNT):
        t = threading.Thread(name=f"c{i}", target=consumer, args=(evt,))
        t.start()
        threads.append(t)
    p = threading.Thread(name='p', target=producer, args=(evt,))
    p.start()
    threads.append(p)
    for t in threads:
        t.join()
        
main()

p: wait 2s
p: Making resource available
c1: Resource is available to consumer
c3: Resource is available to consumer
c2: Resource is available to consumer
c0: Resource is available to consumer


> 多进程版本

In [73]:
%%writefile src/event.py
import time
import multiprocessing 
from random import randint

CONSUMER_COUNT = 4

def consumer(evt):
    TIMEOUT = 3
    t = multiprocessing.current_process()
    evt.wait(TIMEOUT)
    print(f'{t.name}: Resource is available to consumer')
        
def producer(evt):
    t = multiprocessing.current_process()
    print(f'{t.name}: wait 2s')
    time.sleep(2)
    print(f'{t.name}: Making resource available')
    evt.set()  # 设置事件唤醒消费者


if __name__ == '__main__':
    evt = multiprocessing.Event() 
    processes = []
    for i in range(CONSUMER_COUNT):
        t = multiprocessing.Process(name=f"c{i}", target=consumer, args=(evt,))
        t.start()
        processes.append(t)
    p = multiprocessing.Process(name='p', target=producer, args=(evt,))
    p.start()
    processes.append(p)
    for t in processes:
        t.join()

Overwriting src/event.py


In [74]:
!python src/event.py

p: wait 2s
p: Making resource available
c3: Resource is available to consumer
c1: Resource is available to consumer
c2: Resource is available to consumer
c0: Resource is available to consumer


### 条件Condition

条件用于信号通信,它在本质上合并了`Event`和`Lock`的功能.初始化时也可以传入一个锁对象,因此多个Condition可以共用同一个锁.除了拥有锁的所有接口外,还有接口:


+ `notify(n=1)` 释放出通知,让使用相同Condition对象的几个过程知道这个条件已被激活

+ `notify_all()` 释放出通知,让使用相同Condition对象的所有过程知道这个条件已被激活

+ `wait()` 等待使用相同Condition对象的过程的通知.

+ `wait_for(predicate)`相当于
    ```python
    while not predicate():
        cv.wait()
    ```

条件在线程,进程,协程中的使用的模块并不一样:

+ 协程--`asyncio.Condition(lock=None)`

+ 线程--`threading.Condition(lock=None)`

+ 进程--`multiprocessing.Condition([lock])`


通常用法是一个过程等待特定条件,而另一个过程发出特定条件满足的信号,和事件不同的点在于条件可以控制让多少监听者获得信号.

最好说明的例子就是<生产者/多消费者>模型:

> 协程版本

In [43]:
import random
import asyncio


CONSUMER_COUNT = 4

async def consumer(cond):
    t = asyncio.current_task()
    async with cond:
        await cond.wait()  # wait()方法创建了一个名为waiter的锁，
        #并且设置锁的状态为locked。这个waiter锁用于线程间的通讯
        print(f'{t.get_name()}: Resource is available to consumer')
        
async def producer(cond):
    t = asyncio.current_task()
    for i in range(CONSUMER_COUNT):
        # 一次唤醒一个消费者
        print(f'{t.get_name()}: wait 2s')
        await asyncio.sleep(2)
        async with cond:
            print(f'{t.get_name()}: Making resource available')
            cond.notify(1)  # 释放waiter锁，唤醒消费者
        
        
async def main():
    condition = asyncio.Condition()
    cors = []
    for i in range(CONSUMER_COUNT):
        c = asyncio.create_task(consumer(condition), name=f'c{i}')
        cors.append(c)
    
    p = asyncio.create_task(producer(condition), name='p')
    cors.append(p)
    for task in asyncio.as_completed(cors):
        await task

await main()

p: wait 2s
p: Making resource available
p: wait 2s
c0: Resource is available to consumer
p: Making resource available
p: wait 2s
c1: Resource is available to consumer
p: Making resource available
p: wait 2s
c2: Resource is available to consumer
p: Making resource available
c3: Resource is available to consumer


> 线程版本

In [46]:
import time
import threading

CONSUMER_COUNT = 4

def consumer(cond):
    t = threading.current_thread()
    with cond:
        cond.wait()  # wait()方法创建了一个名为waiter的锁，
        #并且设置锁的状态为locked。这个waiter锁用于线程间的通讯
        print(f'{t.name}: Resource is available to consumer')

def producer(cond):
    t = threading.current_thread()
    for i in range(CONSUMER_COUNT):
        # 一次唤醒一个消费者
        print(f'{t.name}: wait 2s')
        time.sleep(2)
        with cond:
            print(f'{t.name}: Making resource available')
            cond.notify(1)  # 释放waiter锁，唤醒消费者
def main():
    condition = threading.Condition()
    threads = []
    for i in range(CONSUMER_COUNT):
        c = threading.Thread(name=f'c{i}', target=consumer, args=(condition,))
        c.start()
        threads.append(c)
        
    p = threading.Thread(name='p', target=producer, args=(condition,))
    p.start()
    threads.append(p)
    for t in threads:
        t.join()

main()

p: wait 2s
p: Making resource available
p: wait 2s
c0: Resource is available to consumer
p: Making resource available
p: wait 2s
c1: Resource is available to consumer
p: Making resource available
p: wait 2s
c2: Resource is available to consumer
p: Making resource available
c3: Resource is available to consumer


> 进程版本

In [51]:
%%writefile src/cond.py
import time
import multiprocessing

CONSUMER_COUNT = 4

def consumer(cond):
    t = multiprocessing.current_process()
    with cond:
        cond.wait()  # wait()方法创建了一个名为waiter的锁，
        #并且设置锁的状态为locked。这个waiter锁用于线程间的通讯
        print(f'{t.name}: Resource is available to consumer')
def producer(cond):
    t = multiprocessing.current_process()
    for i in range(CONSUMER_COUNT):
        # 一次唤醒一个消费者
        print(f'{t.name}: wait 2s')
        time.sleep(2)
        with cond:
            print(f'{t.name}: Making resource available')
            cond.notify()  # 释放waiter锁，唤醒消费者
        
if __name__=='__main__':
    condition = multiprocessing.Condition()
    processes = []
    for i in range(CONSUMER_COUNT):
        c = multiprocessing.Process(name=f'c{i}', target=consumer, args=(condition,))
        c.start()
        processes.append(c)

    p = multiprocessing.Process(name='p', target=producer, args=(condition,))
    p.start()
    processes.append(p)
    for t in processes:
        t.join()

Overwriting src/cond.py


In [52]:
!python src/cond.py

p: wait 2s
p: Making resource available
p: wait 2s
c1: Resource is available to consumer
p: Making resource available
p: wait 2s
c0: Resource is available to consumer
p: Making resource available
p: wait 2s
c2: Resource is available to consumer
p: Making resource available
c3: Resource is available to consumer


## 并行通信

并行程序间需要护传消息,一些同步原语在一定程度上可以达到相当于传递信号消息的作用,但如果是复杂消息就无能为了,多数编程语言都使用队列进行真正意义上的并行通信,python也不例外.

同时队列也是生产者消费者模式最常见使用的工具

队列的接口有:

+ `qsize()`返回队列的大致大小

+ `empty()`如果队列为空返回True

+ `full()`如果队列满了,则返回空

+ `put(item, block=True, timeout=None)`将元素放入队列,协程中是协程

+ `put_nowait(item)`立即将元素放入队列

+ `get(block=True, timeout=None)`获取元素,并且在队列中删除该元素,协程中是协程

+ `get_nowait()`立即获取元素,并且在队列中删除该元素

+ `task_done()`表明以前入队的任务是否已经完成。

+ `join()`阻塞直到队列中的所有项目都被获取和处理.协程中是协程


常见的队列有两种:

+ `Queue`先进先出队列

+ `LifoQueue`先进后出队列

+ `PriorityQueue`优先权队列,放入的元素必须是Tuple[int,Any],第一位就是权重

对不同方式使用的队列为:

+ 协程--`asyncio.Queue(maxsize=0)`/`asyncio.PriorityQueue(maxsize=0)`/`asyncio.LifoQueue(maxsize=0)`

+ 线程--`queue.Queue(maxsize=0)`/`queue.LifoQueue(maxsize=0)`/`queue.PriorityQueue(maxsize=0)`

+ 进程--`multiprocessing.Queue(maxsize)`/`queue.LifoQueue(maxsize=0)`/`queue.PriorityQueue(maxsize=0)`


依然用生产消费模式做例子

> 协程版本

In [101]:
import asyncio
import random

q = asyncio.PriorityQueue(10)

def double(n):
    return n * 2

async def producer(queue, n):
    count = 0
    while True:
        if count > 5:
            break
        pri = randint(0, 100)
        print(f'put :{pri}')
        await queue.put((pri, double, pri))  # (priority, func, args)
        count += 1



async def consumer(queue):
    while True:
        pri, task, arg = await queue.get()
        print(f'[PRI:{pri}] {arg} * 2 = {task(arg)}')
        await asyncio.sleep(random.random())
        queue.task_done()


async def main(n=10):
    # schedule the consumer
    consume = asyncio.ensure_future(consumer(q))
    # run the producer and wait for completion
    await producer(q, n)
    # wait until the consumer has processed all items
    await q.join()
    # the consumer is still awaiting for an item, cancel it
    consume.cancel()

await main()

put :38
put :9
put :8
put :29
put :73
put :83
[PRI:8] 8 * 2 = 16
[PRI:9] 9 * 2 = 18
[PRI:29] 29 * 2 = 58
[PRI:38] 38 * 2 = 76
[PRI:73] 73 * 2 = 146
[PRI:83] 83 * 2 = 166


> 线程版本

In [100]:
import time
import threading
from random import randint
from queue import PriorityQueue

q = PriorityQueue(10)

def double(n):
    return n * 2

def producer(q):
    count = 0
    while True:
        if count > 5:
            break
        pri = randint(0, 100)
        print(f'put :{pri}')
        q.put((pri, double, pri))  # (priority, func, args)
        count += 1

def consumer(q):
    while True:
        if q.empty():
            break
        pri, task, arg = q.get()
        print(f'[PRI:{pri}] {arg} * 2 = {task(arg)}')
        q.task_done()
        time.sleep(0.1)

def main():
    threads = []
    p= threading.Thread(target=producer,args=(q,))
    p.start()
    threads.append(p)
    time.sleep(1)
    c = threading.Thread(target=consumer,args=(q,))
    c.start()
    threads.append(c)
    for t in threads:
        t.join()

main()

put :69
put :17
put :19
put :80
put :43
put :94
[PRI:17] 17 * 2 = 34
[PRI:19] 19 * 2 = 38
[PRI:43] 43 * 2 = 86
[PRI:69] 69 * 2 = 138
[PRI:80] 80 * 2 = 160
[PRI:94] 94 * 2 = 188


> 进程版本

In [98]:
%%writefile src/queue_test.py
import time
from multiprocessing import Process, JoinableQueue
from random import randint

q = JoinableQueue(10)

def double(n):
    return n * 2

def producer(q):
    print("producer started")
    count = 0
    while True:
        if count > 5:
            break
        pri = randint(0, 100)
        print(f'put :{pri}')
        q.put((pri, double, pri))  # (priority, func, args)
        count += 1
        
def consumer(q):
    print("consumer started")
    while True:
        if q.empty():

            break
        pri, task, arg = q.get()
        print(f'[PRI:{pri}] {arg} * 2 = {task(arg)}')
        q.task_done()
        time.sleep(0.1)
        
if __name__=='__main__':
    processes = []
    p = Process(target=producer,args=(q,))
    p.start()
    processes.append(p)
    time.sleep(1)
    c = Process(target=consumer,args=(q,))
    c.start()
    processes.append(c)
    for t in processes:
        t.join()

Overwriting src/queue_test.py


In [99]:
!python src/queue_test.py

producer started
put :31
put :87
put :40
put :78
put :82
put :63
consumer started
[PRI:31] 31 * 2 = 62
[PRI:87] 87 * 2 = 174
[PRI:40] 40 * 2 = 80
[PRI:78] 78 * 2 = 156
[PRI:82] 82 * 2 = 164
[PRI:63] 63 * 2 = 126


## 共享数据

不同的协程/线程/进程间除了传递数据,往往也需要共享数据来实现一些功能,如果是在同一个进程中,通常我们直接使用`全局变量`+`锁`的方式,如果是多进程间则需要借助[multiprocessing.Manager](https://blog.hszofficial.site/TutorialForPython/#/%E8%AF%AD%E6%B3%95%E7%AF%87/%E6%B5%81%E7%A8%8B%E6%8E%A7%E5%88%B6/%E5%A4%9A%E8%BF%9B%E7%A8%8B?id=%e9%ab%98%e7%ba%a7%e5%85%b1%e4%ba%abmultiprocessingmanager)+`锁`的方式来实现

## 有状态上下文

并行程序的有状态上下文指的是一个有进入有退出的范围在这个范围内数据是独立的.举个例子,如果我们写一个http服务,每个请求进来,请求都不同,响应也都不同,我们可以使用有状态上下文管理请求和响应数据.著名的http框架[flask](https://github.com/pallets/flask)就使用了这样的设计.

python中有标准库[contextvars](https://docs.python.org/zh-cn/3.10/library/contextvars.html)用来管理和操作有状态上下文.需要注意它的使用是有限制的

+ 只有协程和线程可以使用
+ 协程自带,可以隐式的使用;线程必须显式使用
+ 必须定义在顶层模块中

### 协程中的有状态上下文

在使用协程时只需要使用类`contextvars.ContextVar`即可,通过它在顶层模块中定义一个上下文变量实例.然后在协程任务的外层开始位置调用实例的`.set(value)`方法设置当前协程的状态上下文初始值;在末尾位置调用实例的`.reset(token)`方法进行重置.之后只要在这个协程中你就可以保持这个上下文变量对象独立了.

In [131]:
import asyncio
from contextvars import ContextVar

var: ContextVar[int] = ContextVar('var', default=42)
CONSUMER_COUNT = 5

async def inner(n):
    t = asyncio.current_task()
    print(f"[{t.get_name()}]inner get var {var.get()}")
    await asyncio.sleep(1)
    var.set(n**2)
    print(f"[{t.get_name()}]inner set var {var.get()}")

async def consumer(n):
    t = asyncio.current_task()
    token = var.set(n)
    print(f"[{t.get_name()}]outer set var {var.get()}")
    await inner(n)
    print(f"[{t.get_name()}]outer get var {var.get()}")
    var.reset(token)
    
async def main():
    cors = []
    for i in range(CONSUMER_COUNT):
        c = asyncio.create_task(consumer(i), name=f'c{i}')
        cors.append(c)
    for task in asyncio.as_completed(cors):
        await task
    
await main()

[c0]outer set var 0
[c1]outer set var 1
[c2]outer set var 2
[c3]outer set var 3
[c4]outer set var 4
[c0]inner get var 0
[c0]inner set var 0
[c0]outer get var 0
[c1]inner get var 1
[c1]inner set var 1
[c1]outer get var 1
[c2]inner get var 2
[c2]inner set var 4
[c2]outer get var 4
[c3]inner get var 3
[c3]inner set var 9
[c3]outer get var 9
[c4]inner get var 4
[c4]inner set var 16
[c4]outer get var 16


### 多线程中使用有状态上下文

在多线程中使用有状态上下文相对麻烦些,我们需要通过接口`contextvars.copy_context`为每个线程复制一份当前的有状态上下文对象`contextvars.Context`,然后使用在启动线程时使用有状态上下文对象的`.run(func,*args,**kwargs)`接口将这个上下文带进需要执行的线程中.

In [132]:
import threading
import time
from contextvars import ContextVar,copy_context

var: ContextVar[int] = ContextVar('var', default=42)
CONSUMER_COUNT = 5

def inner(n):
    t = threading.current_thread()
    print(f"[{t.name}]inner get var {var.get()}")
    time.sleep(1)
    var.set(n**2)
    print(f"[{t.name}]inner set var {var.get()}")

def consumer(n):
    t = threading.current_thread()
    token = var.set(n)
    print(f"[{t.name}]outer set var {var.get()}")
    inner(n)
    print(f"[{t.name}]outer get var {var.get()}")
    var.reset(token)
    
def main():
    threads = []
    for i in range(CONSUMER_COUNT):
        ctx = copy_context()
        c = threading.Thread(target=partial(ctx.run,consumer,i),name=f'c{i}')
        c.start()
        threads.append(c)
    for t in threads:
        t.join()
    
main()

[c0]outer set var 0[c1]outer set var 1
[c1]inner get var 1

[c0]inner get var 0
[c2]outer set var 2
[c2]inner get var 2
[c3]outer set var 3
[c3]inner get var 3
[c4]outer set var 4
[c4]inner get var 4
[c1]inner set var 1[c2]inner set var 4
[c2]outer get var 4
[c4]inner set var 16
[c4]outer get var 16

[c1]outer get var 1
[c3]inner set var 9
[c3]outer get var 9
[c0]inner set var 0
[c0]outer get var 0
